In [41]:
from bs4 import BeautifulSoup

import requests
import re
import math
import pandas as pd
import time
def no_results(text):
    text = text.lower()
    if((text.find("no")>-1) & (text.find("matching")>-1 )& (text.find("results")>-1)):
        return True ;
    else:
        return False;
    
def get_max_pag_num(text):
    index = text.find("of")
        
    return math.ceil(int(text[index+3:])/9)

def get_date(numbers,title):
    output = [] ; 
    for num in numbers:
        if((len(num) == 4 )&(int(num[:2])>18 )& (int(num[:2])<21)):
            #print("found date = " + num)
            output.append(num); 
    
    if((len(output)>1) | (len(output)==0)):
        print_trace(title,'get_date'); 
        return -1 ;
    else:
        return output[0] ; 
            
#make sure numbers is a list of string
#create check for the numbers in list before
def get_section(title):
    index_section = title.lower().find("section"); 
    index_in = title.lower().find("in")
    if(index_section > -1):
        if(index_in > index_section +4):
            #something
            return title[index_section:index_in-1];
        else:
            print_trace(title, "get_section - Cannot find in keyword after section")
            return None ;
    else:
        return None ;
'''
#get section number  by searching all the numbers in the title.
def get_section(title,numbers):
    
    index_section = (title.lower()).find("section")
    if(index_section > -1):
        answer = -1; 
        distance = 999999; 
        for num in numbers:
            temp_distance = title.lower().find(num) -index_section ;
            if(temp_distance < distance):
                answer = num ; 
                distance = temp_distance ;
        
        return answer;
    else:
        return  None
   ''' 
def print_trace(title,func=""):
    print("Error in Func = "+func);
    print("Base_url : " + base_url); 
    print("Title :" + title); 
    print("year start = " + year_start); 
    print("year end = " + year_end);

def get_browse_url(base_url,y_s,y_e,i):
    temp =114
    base_url = base_url[:temp] + str(int(y_s)+i)+ base_url[temp+4:]
    temp =139
    base_url = base_url[:temp] + str(int(y_e)+i)+ base_url[temp+4:]
    return base_url

basic_url = "https://indiankanoon.org"
base_url = "https://indiankanoon.org/search/?formInput=title%3A%28Amendment%29%20sortby%3A%20mostrecent%20fromdate%3A%2001-01-1953%20todate%3A%2031-12-1953%20doctypes%3Alaws%2Cjudgments%2C"
year_start = '2014' ; 
year_end = '2019' ;

df = pd.read_pickle("andha_panda")
loop_index = len(df.index); 
#loop_index =0; 

print("Loop Index = "+str(loop_index))
#df = pd.DataFrame(columns=['Title', 'Section', 'URL','Act_Name','Date','Document Num','CompleteAct_URL','CompleteAct_DocNum','Headline'])
for i in range(int(year_end)-int(year_start)):#for each advance search page
    browse_adv_url = get_browse_url(base_url,year_start,year_start,i);
    print("Browse date = "+str(int(year_start)+i))
    response  = requests.get(browse_adv_url)
    if(response.ok==False):
        print("Http request failed to : "+base_url)
        raise

    
    browse_amendments = response.text
    browse_page = BeautifulSoup(browse_amendments);
    temp_num_result_tag =browse_page.find_all("div", {"class": "results_middle"})
    page_results_num_text = temp_num_result_tag[0].find_all("b")[0].text
    max_page_num  = -1;
    print("Num Pages = "+page_results_num_text)
    if(no_results(page_results_num_text) == False ):
        max_page_num = get_max_pag_num(page_results_num_text)
    else:
        print_trace("","no_results found in url = "+browse_adv_url) ;
        continue 
    print( "max_page_num = "+str(max_page_num));
    if(max_page_num>40):
        max_page_num=41 ;
    
    for page_num in range(max_page_num):
        time.sleep(2)
        print("Browsing page = "+str(page_num));
        response  = requests.get(browse_adv_url+"&pagenum="+str(page_num))
        if(response.ok==False):
            print("Http request failed to : "+base_url)
            raise
        browse_results_pp = BeautifulSoup(response.text);
        temp_num_result_tag_pp =browse_results_pp.find_all("div", {"class": "results_middle"})
        page_results_num_text_pp = temp_num_result_tag_pp[0].find_all("b")[0].text
        if(no_results(page_results_num_text_pp) == True ):
            print_trace("","Error :: no_results found in url = "+browse_adv_url+"&pagenum="+str(page_num)) ;
            continue 
        results_div = browse_results_pp.find_all("div", {"class": "result"})
        for result_tag in results_div:
            time.sleep(2)
            print('############################################')
            #title
            title = result_tag.find_all("a", {"class": "result_url"})[0].text
            if(len(title) == 0):
                print("","Error:: Title not found url = "+browse_adv_url+"&pagenum="+str(page_num));
            print("Title = "+title);
            print("Lopp index  = ",loop_index)

            if((title.lower().find('amendment')<0)|title.lower().find('act')<0 ):
                continue
            df.loc[loop_index,'Title']= title ;
            #title end
            
            #document url
            url = basic_url+result_tag.find_all("a", {"class": "result_url"})[0].get('href') 
            #print("URL = "+url);
            df.loc[loop_index,'URL']= url ;

            #document url end
            
            #headline
            headline = result_tag.find_all("div",{"class":"headline"})[0].text
            #print("Headline = "+headline[:20]);
            df.loc[loop_index,'Headline']= headline ;

            #headline end
            
            #section
            section = get_section(title)
            #print("Section = "+str(section));
            df.loc[loop_index,'Section']= section ;

            #section end
            
            #date
            numbers = re.findall('\d+',title)
            date = get_date(numbers,title)
            #print("date = "+date);
            df.loc[loop_index,'Date']= date ;

            #date end
            
            #act extraction
            temp_s = title
            bracket_left = temp_s.lower().find('(')
            bracket_right = temp_s.lower().find(')')
            temp_act = temp_s.replace(' '+temp_s[bracket_left : bracket_right+1],'')

            index_start = temp_act.lower().find('the') ;
            index_end = temp_act.lower().find('act')
            act = temp_act[index_start:index_end+3]
            #print("act = "+act);
            df.loc[loop_index,'Act_Name']= act ;

            #act extraction end
            
            #compete act url extract 
            a = result_tag.find_all('a')
            complete_act_urls_temp = []
            complete_act_url =''
            for a_ in a:
                if((a_.text.lower().find("complete")>-1) & (a_.text.lower().find('act'))):
                    complete_act_urls_temp.append(a_.get('href'))
            if(len(complete_act_urls_temp)==0):
                complete_act_url = '';
            elif (len(complete_act_urls_temp)>1):
                print_trace("","Error:: Complete act number is more than 1 url = "+browse_adv_url+"&pagenum="+page_numstr(page_num))
            else:
                complete_act_url =basic_url + complete_act_urls_temp[0]; 
            #print("Complete act url = "+complete_act_url);
            df.loc[loop_index,'CompleteAct_URL']= complete_act_url ;

            #compete act url extract  end
            
            #compete act & section act document number
            if((section == None) & (complete_act_url =='')):
                temp_url = url ;
                temp_url = temp_url.replace( basic_url+"/docfragment/",'')
                result_act_document_num = temp_url[:temp_url.find('/')]
            else:
                result_act_document_num = url[len(basic_url)+5:len(url)-1] ;
            complete_act_document_num  = None
            if(complete_act_url != ''):
                complete_act_document_num =complete_act_url[len(basic_url)+5:len(complete_act_url)-1] ;
            #print("result_act_document_num = "+result_act_document_num);
            #print("complete_act_document_num = "+str(complete_act_document_num));
            df.loc[loop_index,'CompleteAct_DocNum']= complete_act_document_num ;
            df.loc[loop_index,'Document Num']= result_act_document_num ;

            #complete act & section act document number

            response_result_act  = requests.get(url)
            if(response_result_act.ok==False):
                print("Http request failed to : "+url)
                loop_index = loop_index+1;
                continue
            act_page  = BeautifulSoup(response_result_act.text)
            act_text_div = act_page.find_all("div", {"class": "acts"})
            
            if(len(act_text_div)>1):
                print_trace(title,"more than 1 act tags found in the url :"+url)
            elif(len(act_text_div) ==0):
                print_trace("#############Cannot find act in this page url ="+url);
                continue
            act_text_all_tags = act_text_div[0].find_all(); 
            
            f  = open("Database/"+str(result_act_document_num)+".txt","w+");
            f_html  = open("Database/html_"+str(result_act_document_num)+".html","w+");
            f_html.write(str(act_text_div[0]));
            for tag in act_text_all_tags:
                f.write(tag.text);
                f.write('\n');
                f.write('\n');
            f.close(); 
            
            f_html.close()
            loop_index = loop_index+1;
df.to_pickle("andha_panda")
df.to_csv("andha_output_data")
print("END #")


Loop Index = 2716
Browse date = 2014
Num Pages = 1 - 10 of 23
max_page_num = 3
Browsing page = 0
############################################
Title = Dilavarsinh Khodubha Since ... vs Draft Amendment Granted. ... on 5 December, 2014
Lopp index  =  2716
############################################
Title = Himanshu Amritbhai Patel & 4 vs After The Amendment Is Carried Out on 3 December, 2014
Lopp index  =  2716
############################################
Title = Commissioner Of Central vs Draft Amendment Dated 24.11.2014 ... on 24 November, 2014
Lopp index  =  2716
############################################
Title = Indian Oil Corporation Limited. & ... vs Draft Amendment Is Allowed. ... on 10 November, 2014
Lopp index  =  2716
############################################
Title = Clma No. 10384 Of 2014 (Amendment ... vs Mr. A.S. Gill on 15 September, 2014
Lopp index  =  2716
############################################
Title = Bharatiben Maganlal Trivedi vs Draft Amendment Is Allowed. 

############################################
Title = Mohit Jairambhai Rathod vs Draft Amendment Is Granted. ... on 30 June, 2016
Lopp index  =  2716
############################################
Title = 4 Whether This Case Involves A ... vs Pursuant To The Amendment Carried ... on 28 June, 2016
Lopp index  =  2716
############################################
Title = Rizwan Ahmed Usmani vs Draft Amendment Is Allowed. ... on 22 June, 2016
Lopp index  =  2716
############################################
Title = Jayeshbhai Punjabhai Patel & 2 vs Draft Amendment Is Granted. ... on 15 June, 2016
Lopp index  =  2716
############################################
Title = Mahesh Sevantilal Shah vs Draft Amendment Granted on 8 June, 2016
Lopp index  =  2716
############################################
Title = Rajesh Nanakram Bhodwani vs Draft Amendment Is Allowed. ... on 4 May, 2016
Lopp index  =  2716
############################################
Title = Bhiyabha Abhabha Sumaniya vs Draft Amendment

In [36]:
browse_adv_url

'https://indiankanoon.org/search/?formInput=title%3A%28Amendment%29%20sortby%3A%20mostrecent%20fromdate%3A%2001-01-2004%20todate%3A%2031-12-2004%20doctypes%3Alaws%2Cjudgments%2C'

In [22]:
#extract title
#check for 1 length
title = results_div[0].find_all("a", {"class": "result_url"})[0].text
#check if title contains amendment, if not don't parse it but store it someplace else.
url = basic_url+results_div[0].find_all("a", {"class": "result_url"})[0].get('href') 
headline = results_div[0].find_all("div",{"class":"headline"})[0].text
numbers = re.findall('\d+',title)
section = get_section(title)
#check if section is None
date = get_date(numbers,title)

found date = 1953


In [23]:
#act extraction
temp_s = title
bracket_left = temp_s.lower().find('(')
bracket_right = temp_s.lower().find(')')
temp_act = temp_s.replace(' '+temp_s[bracket_left : bracket_right+1],'')

index_start = temp_act.lower().find('the') ;
index_end = temp_act.lower().find('act')
act = temp_act[index_start:index_end+3]

In [30]:
#for every results_div
a = results_div[1].find_all('a')
complete_act_url = []
for a_ in a:
    if((a_.text.lower().find("complete")>-1) & (a_.text.lower().find('act'))):
        complete_act_url.append(a_.get('href'))
#check for complete_act_url length

In [33]:
response  = requests.get(url)
if(response.ok==False):
    print("Http request failed to : "+base_url)
    raise
act_page  = BeautifulSoup(response.text)

In [41]:
act_text_div = act_page.find_all("div", {"class": "acts"})

In [42]:
if(len(act_text_div)>1):
    print_trace(title,"more than 1 act tags found in the url :"+url)

In [54]:
act_text_all_tags = act_text_div[0].find_all(); 
#parse all the tags and save it in a file _ Also save the act_text_all_tags in a seperate file
#do this same for the complete act url
#save the html part of each of these acts and complete acts

<div class="article" id="9"> <a href="/doc/1283412/">9.</a>  Ordinance 3 of  1946  not to apply to certain high denomination bank notes.- Nothing contained in the High Denomination Bank Notes (Demonetisation) Ordinance,  1946  (3 of  1946  ) shall apply to any bank note of the denominational value of five hundred rupees, one thousand rupees or ten thousand rupees issued after the commencement of this Act, but no such bank note issued before the 13th day of January,  1946  (3 of  1946  ) shall be legal tender in payment or on account of the amount expressed therein at any place in India.</div>